In [2]:
#https://arshren.medium.com/different-graph-neural-network-implementation-using-pytorch-geometric-23f5bf2f3e9f

In [24]:
import numpy as np
import pandas as pd
import os
import torch_geometric.transforms as T
from torch_geometric.datasets import Planetoid
import matplotlib.pyplot as plt
import torch
import torch.nn.functional as F
from torch_geometric.nn import GCNConv
import torch_geometric
from torch.nn import Parameter
from torch_geometric.utils.convert import to_networkx
import networkx as nx
import urllib.request
import tarfile
from torch_geometric.nn import GraphSAGE
from torch_geometric.nn import SAGEConv
from torch_geometric.nn import GATConv
from torch_geometric.loader import NeighborLoader
from torch_geometric.utils import to_networkx
import pdb

In [2]:
# Load the PubMMed dataset
dataset = Planetoid(root='.', name="Pubmed")
data = dataset[0]

In [3]:
# view the dataset details
# Print information about the dataset
print(f'Dataset: {dataset}')
print('-------------------')
print(f'Number of graphs: {len(dataset)}')
print(f'Number of nodes and features: {data.x.shape}')
print(f'Number of features: {dataset.num_features}')
print(f'Number of classes: {dataset.num_classes}')


Dataset: Pubmed()
-------------------
Number of graphs: 1
Number of nodes and features: torch.Size([19717, 500])
Number of features: 500
Number of classes: 3


In [4]:
# Print information about the graph
print(f'\nGraph:')
print('------')
print(f'Training nodes: {sum(data.train_mask).item()}')
print(f'Evaluation nodes: {sum(data.val_mask).item()}')
print(f'Test nodes: {sum(data.test_mask).item()}')
print(f'Edges are directed: {data.is_directed()}')
print(f'Graph has isolated nodes: {data.has_isolated_nodes()}')
print(f'Graph has loops: {data.has_self_loops()}')


Graph:
------
Training nodes: 60
Evaluation nodes: 500
Test nodes: 1000
Edges are directed: False
Graph has isolated nodes: False
Graph has loops: False


In [5]:
#set the device
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [14]:
train_loader = NeighborLoader(
    data,
    num_neighbors=[5, 10],
    batch_size=16,
    input_nodes=data.train_mask,
)
'''
Graph Convolutional Network
GCN takes graphs as an input and applies convolution operations over the graph
'''
class GCN(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = GCNConv(dataset.num_node_features, 16)
        self.conv2 = GCNConv(16, dataset.num_classes)
        self.optimizer = torch.optim.Adam(self.parameters(), lr=0.005, weight_decay=5e-4)
    def forward(self, x, edge_index):
        # x: Node feature matrix 
        # edge_index: Graph connectivity matrix 
        #x, edge_index = data.x, data.edge_index

        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = F.dropout(x, training=self.training)
        x = self.conv2(x, edge_index)
        return x, F.log_softmax(x, dim=1)
    
'''
Graph SAGE: SAmpling and aggreGatE, 
Samples only a subset of neighboring nodes at different depth layers, 
and then the aggregator takes neighbors of the previous layers and aggregates them
'''
class GraphSAGE(torch.nn.Module):
  """GraphSAGE"""
  def __init__(self, dim_in, dim_h, dim_out):
    super().__init__()
    self.sage1 = SAGEConv(dim_in, dim_h*2)
    self.sage2 = SAGEConv(dim_h*2, dim_h)
    self.sage3 = SAGEConv(dim_h, dim_out)
    self.optimizer = torch.optim.Adam(self.parameters(),
                                      lr=0.01,
                                      weight_decay=5e-4)

  def forward(self, x, edge_index):
    h = self.sage1(x, edge_index)
    h = torch.relu(h)
    h = F.dropout(h, p=0.5, training=self.training)
    h = self.sage2(h, edge_index)
    h = torch.relu(h)
    h = F.dropout(h, p=0.2, training=self.training)
    h = self.sage3(h, edge_index)
    return h, F.log_softmax(h, dim=1)
'''
GAT- uses Attention stratgey
compute the hidden representations of each node in the Graph by attending 
over its neighbors using a self-attention strategy
'''
class GAT(torch.nn.Module):
    def __init__(self):
        super(GAT, self).__init__()
        self.hid = 8
        self.in_head = 8
        self.out_head = 1
        
        self.conv1 = GATConv(dataset.num_features, self.hid, heads=self.in_head, dropout=0.6)
        self.conv2 = GATConv(self.hid*self.in_head, dataset.num_classes, concat=False,
                             heads=self.out_head, dropout=0.6)
        self.optimizer = torch.optim.Adam(self.parameters(), lr=0.005, weight_decay=5e-4)

    def forward(self,x, edge_index):
        
        # Dropout before the GAT layer is used to avoid overfitting
        x = F.dropout(x, p=0.6, training=self.training)
        x = self.conv1(x, edge_index)
        x = F.elu(x)
        x = F.dropout(x, p=0.6, training=self.training)
        x = self.conv2(x, edge_index)
        return x,F.log_softmax(x, dim=1)


def accuracy(pred_y, y):
    """Calculate accuracy."""
    return ((pred_y == y).sum() / len(y)).item()


In [29]:
def train(model, data, epochs):
    criterion = torch.nn.CrossEntropyLoss()
    optimizer = model.optimizer
    
    model.train()
    for epoch in range(epochs+1):
      total_loss = 0
      acc = 0
      val_loss = 0
      val_acc = 0

      # Train on batches
      for batch in train_loader:
        optimizer.zero_grad()
        
        _, out = model(batch.x, batch.edge_index)
        loss = criterion(out[batch.train_mask], batch.y[batch.train_mask])
        total_loss += loss
        acc += accuracy(out[batch.train_mask].argmax(dim=1), 
                        batch.y[batch.train_mask])
        loss.backward()
        optimizer.step()

        # Validation
        val_loss += criterion(out[batch.val_mask], batch.y[batch.val_mask])
        val_acc += accuracy(out[batch.val_mask].argmax(dim=1), 
                            batch.y[batch.val_mask])

      # Print metrics every 10 epochs
      if(epoch % 10 == 0):
          print(f'Epoch {epoch:>3} | Train Loss: {total_loss/len(train_loader):.3f} '
                f'| Train Acc: {acc/len(train_loader)*100:>6.2f}% | Val Loss: '
                f'{val_loss/len(train_loader):.2f} | Val Acc: '
                f'{val_acc/len(train_loader)*100:.2f}%')
          
def test(model, data):
    """Evaluate the model on test set and print the accuracy score."""
    model.eval()
    _, out = model(data.x, data.edge_index)
    acc = accuracy(out.argmax(dim=1)[data.test_mask], data.y[data.test_mask])
    return acc

In [33]:
data.edge_index.shape, data.x.shape

(torch.Size([2, 88648]), torch.Size([19717, 500]))

In [30]:
# Create GCN
gcn = GCN().to(device)
print(gcn)

# Train GCN
train(gcn, dataset, 200)

# Test GCN
print(f'\nGCN test accuracy: {test(gcn, data)*100:.2f}%\n')

GCN(
  (conv1): GCNConv(500, 16)
  (conv2): GCNConv(16, 3)
)
Epoch   0 | Train Loss: 1.110 | Train Acc:  32.25% | Val Loss: 1.10 | Val Acc: 32.64%
Epoch  10 | Train Loss: 0.819 | Train Acc:  86.64% | Val Loss: 0.80 | Val Acc: 83.33%
Epoch  20 | Train Loss: 0.536 | Train Acc:  91.28% | Val Loss: 0.66 | Val Acc: 82.77%
Epoch  30 | Train Loss: 0.352 | Train Acc:  92.97% | Val Loss: 0.52 | Val Acc: 76.25%
Epoch  40 | Train Loss: 0.241 | Train Acc:  95.04% | Val Loss: 0.57 | Val Acc: 83.75%
Epoch  50 | Train Loss: 0.207 | Train Acc:  97.92% | Val Loss: 0.41 | Val Acc: 83.04%
Epoch  60 | Train Loss: 0.149 | Train Acc:  98.53% | Val Loss: 0.25 | Val Acc: 90.62%
Epoch  70 | Train Loss: 0.117 | Train Acc:  98.53% | Val Loss: 0.23 | Val Acc: 95.83%
Epoch  80 | Train Loss: 0.102 | Train Acc:  98.53% | Val Loss: 0.47 | Val Acc: 85.42%
Epoch  90 | Train Loss: 0.113 | Train Acc: 100.00% | Val Loss: 0.44 | Val Acc: 86.32%
Epoch 100 | Train Loss: 0.100 | Train Acc: 100.00% | Val Loss: 0.27 | Val Acc: 

In [34]:
def gnn_model_summary(model):
    
    model_params_list = list(model.named_parameters())
    print("----------------------------------------------------------------")
    line_new = "{:>20}  {:>25} {:>15}".format("Layer.Parameter", "Param Tensor Shape", "Param #")
    print(line_new)
    print("----------------------------------------------------------------")
    for elem in model_params_list:
        p_name = elem[0] 
        p_shape = list(elem[1].size())
        p_count = torch.tensor(elem[1].size()).prod().item()
        line_new = "{:>20}  {:>25} {:>15}".format(p_name, str(p_shape), str(p_count))
        print(line_new)
    print("----------------------------------------------------------------")
    total_params = sum([param.nelement() for param in model.parameters()])
    print("Total params:", total_params)
    num_trainable_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
    print("Trainable params:", num_trainable_params)
    print("Non-trainable params:", total_params - num_trainable_params)
gnn_model_summary(gcn)

----------------------------------------------------------------
     Layer.Parameter         Param Tensor Shape         Param #
----------------------------------------------------------------
          conv1.bias                       [16]              16
    conv1.lin.weight                  [16, 500]            8000
          conv2.bias                        [3]               3
    conv2.lin.weight                    [3, 16]              48
----------------------------------------------------------------
Total params: 8067
Trainable params: 8067
Non-trainable params: 0


In [18]:
# Create GraphSAGE
graphsage = GraphSAGE(dataset.num_features, 64, dataset.num_classes).to(device)
print(graphsage)

# Train GraphSAGE
train(graphsage, dataset, 200)

# Test GraphSAGE
print(f'\nGraphSAGE test accuracy: {test(graphsage, data)*100:.2f}%\n')


GraphSAGE(
  (sage1): SAGEConv(500, 128, aggr=mean)
  (sage2): SAGEConv(128, 64, aggr=mean)
  (sage3): SAGEConv(64, 3, aggr=mean)
)
Epoch   0 | Train Loss: 1.175 | Train Acc:  22.70% | Val Loss: 1.13 | Val Acc: 16.67%
Epoch  10 | Train Loss: 0.012 | Train Acc: 100.00% | Val Loss: 0.47 | Val Acc: 77.43%
Epoch  20 | Train Loss: 0.007 | Train Acc: 100.00% | Val Loss: 0.70 | Val Acc: 74.17%
Epoch  30 | Train Loss: 0.072 | Train Acc:  97.22% | Val Loss: 0.64 | Val Acc: 74.90%
Epoch  40 | Train Loss: 0.004 | Train Acc: 100.00% | Val Loss: 0.38 | Val Acc: 88.06%
Epoch  50 | Train Loss: 0.004 | Train Acc: 100.00% | Val Loss: 0.40 | Val Acc: 83.81%
Epoch  60 | Train Loss: 0.008 | Train Acc: 100.00% | Val Loss: 0.62 | Val Acc: 64.91%
Epoch  70 | Train Loss: 0.002 | Train Acc: 100.00% | Val Loss: 0.54 | Val Acc: 76.52%
Epoch  80 | Train Loss: 0.004 | Train Acc: 100.00% | Val Loss: 0.50 | Val Acc: 70.42%
Epoch  90 | Train Loss: 0.007 | Train Acc: 100.00% | Val Loss: 0.73 | Val Acc: 82.08%
Epoch 10

In [35]:
gnn_model_summary(graphsage)

----------------------------------------------------------------
     Layer.Parameter         Param Tensor Shape         Param #
----------------------------------------------------------------
  sage1.lin_l.weight                 [128, 500]           64000
    sage1.lin_l.bias                      [128]             128
  sage1.lin_r.weight                 [128, 500]           64000
  sage2.lin_l.weight                  [64, 128]            8192
    sage2.lin_l.bias                       [64]              64
  sage2.lin_r.weight                  [64, 128]            8192
  sage3.lin_l.weight                    [3, 64]             192
    sage3.lin_l.bias                        [3]               3
  sage3.lin_r.weight                    [3, 64]             192
----------------------------------------------------------------
Total params: 144963
Trainable params: 144963
Non-trainable params: 0


In [19]:
# Create GAT
gat = GAT().to(device)
print(gat)

# Train Graph Attention Network
train(gat, dataset, 200)

# Test GAT
print(f'\nGraph Attention Network test accuracy: {test(gat, data)*100:.2f}%\n')


GAT(
  (conv1): GATConv(500, 8, heads=8)
  (conv2): GATConv(64, 3, heads=1)
)
Epoch   0 | Train Loss: 1.162 | Train Acc:  23.07% | Val Loss: 1.12 | Val Acc: 25.19%
Epoch  10 | Train Loss: 0.752 | Train Acc:  80.04% | Val Loss: 1.03 | Val Acc: 54.58%
Epoch  20 | Train Loss: 0.673 | Train Acc:  80.11% | Val Loss: 1.24 | Val Acc: 46.25%
Epoch  30 | Train Loss: 0.662 | Train Acc:  73.38% | Val Loss: 0.83 | Val Acc: 60.00%
Epoch  40 | Train Loss: 0.513 | Train Acc:  73.37% | Val Loss: 1.07 | Val Acc: 54.85%
Epoch  50 | Train Loss: 0.399 | Train Acc:  86.03% | Val Loss: 0.89 | Val Acc: 64.29%
Epoch  60 | Train Loss: 0.476 | Train Acc:  76.87% | Val Loss: 0.79 | Val Acc: 60.12%
Epoch  70 | Train Loss: 0.393 | Train Acc:  81.19% | Val Loss: 0.88 | Val Acc: 54.10%
Epoch  80 | Train Loss: 0.399 | Train Acc:  84.13% | Val Loss: 0.87 | Val Acc: 59.17%
Epoch  90 | Train Loss: 0.341 | Train Acc:  82.54% | Val Loss: 0.67 | Val Acc: 68.75%
Epoch 100 | Train Loss: 0.379 | Train Acc:  80.21% | Val Loss:

In [36]:
gnn_model_summary(gat)

----------------------------------------------------------------
     Layer.Parameter         Param Tensor Shape         Param #
----------------------------------------------------------------
       conv1.att_src                  [1, 8, 8]              64
       conv1.att_dst                  [1, 8, 8]              64
          conv1.bias                       [64]              64
conv1.lin_src.weight                  [64, 500]           32000
       conv2.att_src                  [1, 1, 3]               3
       conv2.att_dst                  [1, 1, 3]               3
          conv2.bias                        [3]               3
conv2.lin_src.weight                    [3, 64]             192
----------------------------------------------------------------
Total params: 32393
Trainable params: 32393
Non-trainable params: 0
